# 小波包分解九个站点批量计算

In [1]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import tensorflow as tf
import keras
import catboost as cb
from pandas import read_csv
from math import sqrt
from numpy import concatenate
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn import svm
from keras.layers.core import Dense, Activation, Dropout
from sklearn.externals import joblib
%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:472: FutureWarning: Passi

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto(allow_soft_placement = True)
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.5)
config.gpu_options.allow_growth = True
sess0 = tf.InteractiveSession(config = config)

In [3]:
print(os.getcwd()) # 打印当前工作目录

E:\Research Project\DKASC, Alice Springs


In [4]:
os.listdir(os.getcwd())

['.idea',
 '.ipynb_checkpoints',
 '101-Site_DKA-WeatherStation',
 '96-Site_DKA-MasterMeter1',
 'Alice_Springs_2016-2020',
 'Dataset',
 'logs',
 'picture',
 'Predict',
 'Program backups',
 'temp',
 'venv',
 '数据集_小波包分解重构数据.ipynb',
 '数据集_预处理_第二版0702.ipynb',
 '测试小波分解.ipynb',
 '测试模型0622.ipynb',
 '测试模型0704.ipynb',
 '测试模型0704批量.ipynb',
 '测试模型小波包分解0704批量.ipynb',
 '画图.ipynb',
 '画图0704.ipynb',
 '笔记.ipynb',
 '训练模型0622.ipynb',
 '训练模型0702.ipynb',
 '训练模型0703 WPD.ipynb']

In [5]:
os.chdir("./Dataset/testWPD/")

In [6]:
os.listdir()

['WPD_test_Station_1.csv',
 'WPD_test_Station_2.csv',
 'WPD_test_Station_3.csv',
 'WPD_test_Station_4.csv',
 'WPD_test_Station_5.csv',
 'WPD_test_Station_6.csv',
 'WPD_test_Station_7.csv',
 'WPD_test_Station_8.csv',
 'WPD_test_Station_9.csv']

In [7]:
file_chdir = os.getcwd()

In [8]:
file_chdir

'E:\\Research Project\\DKASC, Alice Springs\\Dataset\\testWPD'

In [9]:
filecsv_list = []
for root,dirs,files in os.walk(file_chdir):
    for file in files:
        if os.path.splitext(file)[1] == '.csv':
            filecsv_list.append(file)

In [10]:
filecsv_list

['WPD_test_Station_1.csv',
 'WPD_test_Station_2.csv',
 'WPD_test_Station_3.csv',
 'WPD_test_Station_4.csv',
 'WPD_test_Station_5.csv',
 'WPD_test_Station_6.csv',
 'WPD_test_Station_7.csv',
 'WPD_test_Station_8.csv',
 'WPD_test_Station_9.csv']

In [11]:
testnames=list()
testnames += [('test_%d' % (j)) for j in range(len(filecsv_list))]

In [12]:
testnames

['test_0',
 'test_1',
 'test_2',
 'test_3',
 'test_4',
 'test_5',
 'test_6',
 'test_7',
 'test_8']

In [13]:
for csv in filecsv_list:
    testnames[filecsv_list.index(csv)]=pd.read_csv(csv)

In [14]:
testnames[0]

Timestamp  Year  Month  Day  Hour  Minutes  Temperature(°C)  \
0          2019/1/1 0:00  2019      1    1     0        0        30.020723   
1          2019/1/1 0:05  2019      1    1     0        5        29.958653   
2          2019/1/1 0:10  2019      1    1     0       10        29.884491   
3          2019/1/1 0:15  2019      1    1     0       15        29.831554   
4          2019/1/1 0:20  2019      1    1     0       20        29.750330   
5          2019/1/1 0:25  2019      1    1     0       25        29.532784   
6          2019/1/1 0:30  2019      1    1     0       30        29.333590   
7          2019/1/1 0:35  2019      1    1     0       35        29.115730   
8          2019/1/1 0:40  2019      1    1     0       40        28.895197   
9          2019/1/1 0:45  2019      1    1     0       45        28.870382   
10         2019/1/1 0:50  2019      1    1     0       50        28.844110   
11         2019/1/1 0:55  2019      1    1     0       55        28.717377   
12         2019/1/1 1:00  2019      1    1     1        0        28.702896   
13         2019/1/1 1:05  2019      1    1     1        5        28.701187   
14         2019/1/1 1:10  2019      1    1     1       10        28.662622   
15         2019/1/1 1:15  2019      1    1     1       15        28.844805   
16         2019/1/1 1:20  2019      1    1     1       20        29.389601   
17         2019/1/1 1:25  2019      1    1     1       25        30.003380   
18         2019/1/1 1:30  2019      1    1     1       30        30.191591   
19         2019/1/1 1:35  2019      1    1     1       35        30.349762   
20         2019/1/1 1:40  2019      1    1     1       40        30.253325   
21         2019/1/1 1:45  2019      1    1     1       45        29.668095   
22         2019/1/1 1:50  2019      1    1     1       50        29.148050   
23         2019/1/1 1:55  2019      1    1     1       55        29.482159   
24         2019/1/1 2:00  2019      1    1     2        0        29.759039   
25         2019/1/1 2:05  2019      1    1     2        5        29.592545   
26         2019/1/1 2:10  2019      1    1     2       10        29.456694   
27         2019/1/1 2:15  2019      1    1     2       15        29.235744   
28         2019/1/1 2:20  2019      1    1     2       20        28.912735   
29         2019/1/1 2:25  2019      1    1     2       25        28.703390   
...                  ...   ...    ...  ...   ...      ...              ...   
102858  2019/12/31 21:30  2019     12   31    21       30        32.454887   
102859  2019/12/31 21:35  2019     12   31    21       35        32.379871   
102860  2019/12/31 21:40  2019     12   31    21       40        32.270977   
102861  2019/12/31 21:45  2019     12   31    21       45        32.139584   
102862  2019/12/31 21:50  2019     12   31    21       50        31.968740   
102863  2019/12/31 21:55  2019     12   31    21       55        31.890812   
102864  2019/12/31 22:00  2019     12   31    22        0        31.816862   
102865  2019/12/31 22:05  2019     12   31    22        5        31.748964   
102866  2019/12/31 22:10  2019     12   31    22       10        31.657833   
102867  2019/12/31 22:15  2019     12   31    22       15        31.501961   
102868  2019/12/31 22:20  2019     12   31    22       20        31.367474   
102869  2019/12/31 22:25  2019     12   31    22       25        31.202694   
102870  2019/12/31 22:30  2019     12   31    22       30        31.094898   
102871  2019/12/31 22:35  2019     12   31    22       35        31.025242   
102872  2019/12/31 22:40  2019     12   31    22       40        30.859026   
102873  2019/12/31 22:45  2019     12   31    22       45        30.720379   
102874  2019/12/31 22:50  2019     12   31    22       50        30.648829   
102875  2019/12/31 22:55  2019     12   31    22       55        30.676096   
102876  2019/12/31 23:00  2019     12   31    23        0        30.536377   
102877  2019/12/31 23:05  2019     12

In [15]:
#查看输入数据维度
# print(train.shape)
for tena in testnames:
    print(tena.shape)

(102888, 15)
(102888, 15)
(102888, 15)
(102888, 15)
(102888, 15)
(102888, 15)
(102888, 15)
(102888, 15)
(102888, 15)


In [16]:
for tena in testnames:
    print(tena.head(1))
# test.head(5)

       Timestamp  Year  Month  Day  Hour  Minutes  Temperature(°C)  \
0  2019/1/1 0:00  2019      1    1     0        0        30.020723   

   Relative Humidity(%)  Wind Direction(Degrees)  Daily Rainfall(mm)  \
0             20.125513                 32.21347                 0.0   

   Global Horizontal Radiation(W/m²)  Diffuse Horizontal Radiation(W/m²)    A  \
0                           1.543787                            1.349848  0.0   

     D  Station_1  
0  0.0        0.0  
       Timestamp  Year  Month  Day  Hour  Minutes  Temperature(°C)  \
0  2019/1/1 0:00  2019      1    1     0        0        30.020723   

   Relative Humidity(%)  Wind Direction(Degrees)  Daily Rainfall(mm)  \
0             20.125513                 32.21347                 0.0   

   Global Horizontal Radiation(W/m²)  Diffuse Horizontal Radiation(W/m²)    A  \
0                           1.543787                            1.349848  0.0   

     D  Station_2  
0  0.0        0.0  
       Timestamp  Year

# 数据预处理训练集去异常值

In [17]:
# test.isnull().sum()
for tena in testnames:
    print(tena.isnull().sum())

Timestamp                             0
Year                                  0
Month                                 0
Day                                   0
Hour                                  0
Minutes                               0
Temperature(°C)                       0
Relative Humidity(%)                  0
Wind Direction(Degrees)               0
Daily Rainfall(mm)                    0
Global Horizontal Radiation(W/m²)     0
Diffuse Horizontal Radiation(W/m²)    0
A                                     0
D                                     0
Station_1                             0
dtype: int64
Timestamp                             0
Year                                  0
Month                                 0
Day                                   0
Hour                                  0
Minutes                               0
Temperature(°C)                       0
Relative Humidity(%)                  0
Wind Direction(Degrees)               0
Daily Rainfall(mm)         

In [18]:
# test.columns
for tena in testnames:
    print(tena.columns)

Index(['Timestamp', 'Year', 'Month', 'Day', 'Hour', 'Minutes',
       'Temperature(°C)', 'Relative Humidity(%)', 'Wind Direction(Degrees)',
       'Daily Rainfall(mm)', 'Global Horizontal Radiation(W/m²)',
       'Diffuse Horizontal Radiation(W/m²)', 'A', 'D', 'Station_1'],
      dtype='object')
Index(['Timestamp', 'Year', 'Month', 'Day', 'Hour', 'Minutes',
       'Temperature(°C)', 'Relative Humidity(%)', 'Wind Direction(Degrees)',
       'Daily Rainfall(mm)', 'Global Horizontal Radiation(W/m²)',
       'Diffuse Horizontal Radiation(W/m²)', 'A', 'D', 'Station_2'],
      dtype='object')
Index(['Timestamp', 'Year', 'Month', 'Day', 'Hour', 'Minutes',
       'Temperature(°C)', 'Relative Humidity(%)', 'Wind Direction(Degrees)',
       'Daily Rainfall(mm)', 'Global Horizontal Radiation(W/m²)',
       'Diffuse Horizontal Radiation(W/m²)', 'A', 'D', 'Station_3'],
      dtype='object')
Index(['Timestamp', 'Year', 'Month', 'Day', 'Hour', 'Minutes',
       'Temperature(°C)', 'Relative Humidity(%

In [19]:
len(testnames)

9

In [20]:
for tena in range(0,len(testnames)):
    testnames[tena] = testnames[tena].drop(columns=['Timestamp','Year'])
    print(testnames[tena].columns)

Index(['Month', 'Day', 'Hour', 'Minutes', 'Temperature(°C)',
       'Relative Humidity(%)', 'Wind Direction(Degrees)', 'Daily Rainfall(mm)',
       'Global Horizontal Radiation(W/m²)',
       'Diffuse Horizontal Radiation(W/m²)', 'A', 'D', 'Station_1'],
      dtype='object')
Index(['Month', 'Day', 'Hour', 'Minutes', 'Temperature(°C)',
       'Relative Humidity(%)', 'Wind Direction(Degrees)', 'Daily Rainfall(mm)',
       'Global Horizontal Radiation(W/m²)',
       'Diffuse Horizontal Radiation(W/m²)', 'A', 'D', 'Station_2'],
      dtype='object')
Index(['Month', 'Day', 'Hour', 'Minutes', 'Temperature(°C)',
       'Relative Humidity(%)', 'Wind Direction(Degrees)', 'Daily Rainfall(mm)',
       'Global Horizontal Radiation(W/m²)',
       'Diffuse Horizontal Radiation(W/m²)', 'A', 'D', 'Station_3'],
      dtype='object')
Index(['Month', 'Day', 'Hour', 'Minutes', 'Temperature(°C)',
       'Relative Humidity(%)', 'Wind Direction(Degrees)', 'Daily Rainfall(mm)',
       'Global Horizontal Radia

In [21]:
testnames[0].head()

Month  Day  Hour  Minutes  Temperature(°C)  Relative Humidity(%)  \
0      1    1     0        0        30.020723             20.125513   
1      1    1     0        5        29.958653             20.040049   
2      1    1     0       10        29.884491             20.013313   
3      1    1     0       15        29.831554             20.068233   
4      1    1     0       20        29.750330             20.086567   

   Wind Direction(Degrees)  Daily Rainfall(mm)  \
0                32.213470                 0.0   
1                32.158791                 0.0   
2                32.155808                 0.0   
3                32.178555                 0.0   
4                32.166214                 0.0   

   Global Horizontal Radiation(W/m²)  Diffuse Horizontal Radiation(W/m²)    A  \
0                           1.543787                            1.349848  0.0   
1                           1.373370                            1.244123  0.0   
2                           1.142953                            0.992849  0.0   
3                           0.709036                            0.611533  0.0   
4                           1.058623                            0.814599  0.0   

     D  Station_1  
0  0.0        0.0  
1  0.0        0.0  
2  0.0        0.0  
3  0.0        0.0  
4  0.0        0.0

In [22]:
testnames[0].head()

Month  Day  Hour  Minutes  Temperature(°C)  Relative Humidity(%)  \
0      1    1     0        0        30.020723             20.125513   
1      1    1     0        5        29.958653             20.040049   
2      1    1     0       10        29.884491             20.013313   
3      1    1     0       15        29.831554             20.068233   
4      1    1     0       20        29.750330             20.086567   

   Wind Direction(Degrees)  Daily Rainfall(mm)  \
0                32.213470                 0.0   
1                32.158791                 0.0   
2                32.155808                 0.0   
3                32.178555                 0.0   
4                32.166214                 0.0   

   Global Horizontal Radiation(W/m²)  Diffuse Horizontal Radiation(W/m²)    A  \
0                           1.543787                            1.349848  0.0   
1                           1.373370                            1.244123  0.0   
2                           1.142953                            0.992849  0.0   
3                           0.709036                            0.611533  0.0   
4                           1.058623                            0.814599  0.0   

     D  Station_1  
0  0.0        0.0  
1  0.0        0.0  
2  0.0        0.0  
3  0.0        0.0  
4  0.0        0.0

# keras模型训练

准备数据

In [23]:
for tena in testnames:
    print(tena.head())

   Month  Day  Hour  Minutes  Temperature(°C)  Relative Humidity(%)  \
0      1    1     0        0        30.020723             20.125513   
1      1    1     0        5        29.958653             20.040049   
2      1    1     0       10        29.884491             20.013313   
3      1    1     0       15        29.831554             20.068233   
4      1    1     0       20        29.750330             20.086567   

   Wind Direction(Degrees)  Daily Rainfall(mm)  \
0                32.213470                 0.0   
1                32.158791                 0.0   
2                32.155808                 0.0   
3                32.178555                 0.0   
4                32.166214                 0.0   

   Global Horizontal Radiation(W/m²)  Diffuse Horizontal Radiation(W/m²)    A  \
0                           1.543787                            1.349848  0.0   
1                           1.373370                            1.244123  0.0   
2                           1

# 构建序列

In [24]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
#     n_vars = 1 if type(data) is list else data.shape[1]
    n_vars = list(data)
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('%s(t-%d)' % (j, i)) for j in n_vars]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('%s(t)' % (j) )for j in n_vars]
        else:
            names += [('%s(t+%d)' % (j, i)) for j in n_vars]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [25]:
# test=glocals()
# test += [('test%d' % (j)) for j in range(len(testnames))]

In [26]:
for i in range(0, 9):
    exec("test%d=series_to_supervised(testnames[i],12,1)"%i)


In [27]:
test0.columns

Index(['Month(t-12)', 'Day(t-12)', 'Hour(t-12)', 'Minutes(t-12)',
       'Temperature(°C)(t-12)', 'Relative Humidity(%)(t-12)',
       'Wind Direction(Degrees)(t-12)', 'Daily Rainfall(mm)(t-12)',
       'Global Horizontal Radiation(W/m²)(t-12)',
       'Diffuse Horizontal Radiation(W/m²)(t-12)',
       ...
       'Minutes(t)', 'Temperature(°C)(t)', 'Relative Humidity(%)(t)',
       'Wind Direction(Degrees)(t)', 'Daily Rainfall(mm)(t)',
       'Global Horizontal Radiation(W/m²)(t)',
       'Diffuse Horizontal Radiation(W/m²)(t)', 'A(t)', 'D(t)',
       'Station_1(t)'],
      dtype='object', length=169)

In [ ]:
# for i in range(0, 9):
#     exec("test%d=test%d.drop(test%d.columns[[0,14,15,16,17,18,19,20,21,22,23,24,25,26,28,29,30,31,32,33,34,35,36,37,38,39,40]],axis=1,inplace=True)"%i)

# 重新选择

In [31]:
test0.shape[1]

169

In [32]:
test0.drop(test0.columns[13:test0.shape[1]-1],axis=1,inplace=True)

In [33]:
test1.drop(test1.columns[13:test1.shape[1]-1],axis=1,inplace=True)

In [34]:
test2.drop(test2.columns[13:test2.shape[1]-1],axis=1,inplace=True)

In [35]:
test3.drop(test3.columns[13:test3.shape[1]-1],axis=1,inplace=True)

In [36]:
test4.drop(test4.columns[13:test4.shape[1]-1],axis=1,inplace=True)

In [37]:
test5.drop(test5.columns[13:test5.shape[1]-1],axis=1,inplace=True)

In [38]:
test6.drop(test6.columns[13:test6.shape[1]-1],axis=1,inplace=True)

In [39]:
test7.drop(test7.columns[13:test7.shape[1]-1],axis=1,inplace=True)

In [40]:
test8.drop(test8.columns[13:test8.shape[1]-1],axis=1,inplace=True)

In [41]:
print(test0.shape,test1.shape,test2.shape,test3.shape,test4.shape,test5.shape,test6.shape,test7.shape,test8.shape,)

(102876, 14) (102876, 14) (102876, 14) (102876, 14) (102876, 14) (102876, 14) (102876, 14) (102876, 14) (102876, 14)


In [42]:
test0.columns

Index(['Month(t-12)', 'Day(t-12)', 'Hour(t-12)', 'Minutes(t-12)',
       'Temperature(°C)(t-12)', 'Relative Humidity(%)(t-12)',
       'Wind Direction(Degrees)(t-12)', 'Daily Rainfall(mm)(t-12)',
       'Global Horizontal Radiation(W/m²)(t-12)',
       'Diffuse Horizontal Radiation(W/m²)(t-12)', 'A(t-12)', 'D(t-12)',
       'Station_1(t-12)', 'Station_1(t)'],
      dtype='object')

# 归一化数据

In [43]:
# train1 = train1.values
# train1 = train1.astype('float32')     # 将整型变为float
# scaler = MinMaxScaler(feature_range=(0, 1)) # 数据处理，归一化至0~1之间
# train1 = scaler.fit_transform(train1)

# test1 = test1.values
# test1 = test1.astype('float32')     # 将整型变为float
# scaler = MinMaxScaler(feature_range=(0, 1)) # 数据处理，归一化至0~1之间
# test1 = scaler.fit_transform(test1)

# inds = np.arange(train1.shape[0])
# train1_inds = inds[0:int(train1.shape[0]*0.8)]
# eval1_inds = inds[int(train1.shape[0]*0.8):train1.shape[0]]


# train_X, train_y = train1[train1_inds,:-1], train1[train1_inds,-1:]
# eval_X,eval_y = train1[eval1_inds,:-1], train1[eval1_inds,-1:]
# test_X, test_y = test1[:, :-1], test1[:,-1:]

# # reshape input to be 3D [samples, timesteps, features]
# train_Xl = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
# eval_Xl = eval_X.reshape((eval_X.shape[0],1,eval_X.shape[1]))
# test_Xl = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
# print(train_X.shape, train_y.shape,eval_X.shape,eval_y.shape, test_X.shape, test_y.shape)
# print(train_Xl.shape, train_y.shape,eval_Xl.shape,eval_y.shape, test_Xl.shape, test_y.shape)

0

In [44]:
test0 = test0.values
test0 = test0.astype('float32')     # 将整型变为float
scaler = MinMaxScaler(feature_range=(0, 1)) # 数据处理，归一化至0~1之间
test0 = scaler.fit_transform(test0)

test0_X, test0_y = test0[:, :-1], test0[:,-1:]

# reshape input to be 3D [samples, timesteps, features]
test0_Xl = test0_X.reshape((test0_X.shape[0], 1, test0_X.shape[1]))
print(test0_X.shape, test0_y.shape)
print(test0_Xl.shape, test0_y.shape)

(102876, 13) (102876, 1)
(102876, 1, 13) (102876, 1)


1

In [45]:
test1 = test1.values
test1 = test1.astype('float32')     # 将整型变为float
scaler = MinMaxScaler(feature_range=(0, 1)) # 数据处理，归一化至0~1之间
test1 = scaler.fit_transform(test1)

test1_X, test1_y = test1[:, :-1], test1[:,-1:]

# reshape input to be 3D [samples, timesteps, features]
test1_Xl = test1_X.reshape((test1_X.shape[0], 1, test1_X.shape[1]))
print(test1_X.shape, test1_y.shape)
print(test1_Xl.shape, test1_y.shape)

(102876, 13) (102876, 1)
(102876, 1, 13) (102876, 1)


2

In [46]:
test2 = test2.values
test2 = test2.astype('float32')     # 将整型变为float
scaler = MinMaxScaler(feature_range=(0, 1)) # 数据处理，归一化至0~1之间
test2 = scaler.fit_transform(test2)

test2_X, test2_y = test2[:, :-1], test2[:,-1:]

# reshape input to be 3D [samples, timesteps, features]
test2_Xl = test2_X.reshape((test2_X.shape[0], 1, test2_X.shape[1]))
print(test2_X.shape, test2_y.shape)
print(test2_Xl.shape, test2_y.shape)

(102876, 13) (102876, 1)
(102876, 1, 13) (102876, 1)


3

In [47]:
test3 = test3.values
test3 = test3.astype('float32')     # 将整型变为float
scaler = MinMaxScaler(feature_range=(0, 1)) # 数据处理，归一化至0~1之间
test3 = scaler.fit_transform(test3)

test3_X, test3_y = test3[:, :-1], test3[:,-1:]

# reshape input to be 3D [samples, timesteps, features]
test3_Xl = test3_X.reshape((test3_X.shape[0], 1, test3_X.shape[1]))
print(test3_X.shape, test3_y.shape)
print(test3_Xl.shape, test3_y.shape)

(102876, 13) (102876, 1)
(102876, 1, 13) (102876, 1)


4

In [48]:
test4 = test4.values
test4 = test4.astype('float32')     # 将整型变为float
scaler = MinMaxScaler(feature_range=(0, 1)) # 数据处理，归一化至0~1之间
test4 = scaler.fit_transform(test4)

test4_X, test4_y = test4[:, :-1], test4[:,-1:]

# reshape input to be 3D [samples, timesteps, features]
test4_Xl = test4_X.reshape((test4_X.shape[0], 1, test4_X.shape[1]))
print(test4_X.shape, test4_y.shape)
print(test4_Xl.shape, test4_y.shape)

(102876, 13) (102876, 1)
(102876, 1, 13) (102876, 1)


5

In [49]:
test5 = test5.values
test5 = test5.astype('float32')     # 将整型变为float
scaler = MinMaxScaler(feature_range=(0, 1)) # 数据处理，归一化至0~1之间
test5 = scaler.fit_transform(test5)

test5_X, test5_y = test5[:, :-1], test5[:,-1:]

# reshape input to be 3D [samples, timesteps, features]
test5_Xl = test5_X.reshape((test5_X.shape[0], 1, test5_X.shape[1]))
print(test5_X.shape, test5_y.shape)
print(test5_Xl.shape, test5_y.shape)

(102876, 13) (102876, 1)
(102876, 1, 13) (102876, 1)


6

In [50]:
test6 = test6.values
test6 = test6.astype('float32')     # 将整型变为float
scaler = MinMaxScaler(feature_range=(0, 1)) # 数据处理，归一化至0~1之间
test6 = scaler.fit_transform(test6)

test6_X, test6_y = test6[:, :-1], test6[:,-1:]

# reshape input to be 3D [samples, timesteps, features]
test6_Xl = test6_X.reshape((test6_X.shape[0], 1, test6_X.shape[1]))
print(test6_X.shape, test6_y.shape)
print(test6_Xl.shape, test6_y.shape)

(102876, 13) (102876, 1)
(102876, 1, 13) (102876, 1)


7

In [51]:
test7 = test7.values
test7 = test7.astype('float32')     # 将整型变为float
scaler = MinMaxScaler(feature_range=(0, 1)) # 数据处理，归一化至0~1之间
test7 = scaler.fit_transform(test7)

test7_X, test7_y = test7[:, :-1], test7[:,-1:]

# reshape input to be 3D [samples, timesteps, features]
test7_Xl = test7_X.reshape((test7_X.shape[0], 1, test7_X.shape[1]))
print(test7_X.shape, test7_y.shape)
print(test7_Xl.shape, test7_y.shape)

(102876, 13) (102876, 1)
(102876, 1, 13) (102876, 1)


8

In [52]:
test8 = test8.values
test8 = test8.astype('float32')     # 将整型变为float
scaler = MinMaxScaler(feature_range=(0, 1)) # 数据处理，归一化至0~1之间
test8 = scaler.fit_transform(test8)

test8_X, test8_y = test8[:, :-1], test8[:,-1:]

# reshape input to be 3D [samples, timesteps, features]
test8_Xl = test8_X.reshape((test8_X.shape[0], 1, test8_X.shape[1]))
print(test8_X.shape, test8_y.shape)
print(test8_Xl.shape, test8_y.shape)

(102876, 13) (102876, 1)
(102876, 1, 13) (102876, 1)


# SVM model

In [53]:
os.chdir("../")

In [54]:
print(os.getcwd()) # 打印当前工作目录

E:\Research Project\DKASC, Alice Springs\Dataset


In [55]:
os.listdir()

['DKA.Station_1-9.csv',
 'NaN_DKA.Station.csv',
 'raw_DKA.Station_1-9.csv',
 'test',
 'test0702',
 'testdata.csv',
 'testWPD',
 'test_1A.csv',
 'train',
 'train0702',
 'traindata.csv',
 'trainWPD',
 'train_1A.csv',
 '测试集8个站点.csv',
 '训练集8个站点.csv']

# BP model

In [56]:
print(os.getcwd()) # 打印当前工作目录

E:\Research Project\DKASC, Alice Springs\Dataset


In [57]:
#模型加载
from keras.models import load_model
modelbp = load_model('../logs/BP0702WPD.h5') 

# DNN model

In [58]:
#模型加载
modeldnn = load_model('../logs/DNN0702WPD.h5') 

# LSTM model

In [59]:
#模型加载
modellstm = load_model('../logs/LSTM0702WPD.h5') 

# 结果对比

反归一化

In [60]:
def inv(testPredictML,test_X,test_y):
    # invert scaling for forecast
    inv_testPredictML = concatenate((test_X[:,:],testPredictML), axis=1)
    inv_testPredictML = scaler.inverse_transform(inv_testPredictML)
    testPredictML = inv_testPredictML[:,-1:]
    # invert scaling for actual
    test_y = test_y.reshape((len(test_y), 1))
    inv_test = concatenate((test_X[:,:],test_y), axis=1)
    inv_test = scaler.inverse_transform(inv_test)
    testReal = inv_test[:,-1:]
    return testPredictML,testReal,inv_test

In [61]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred - y_true) / y_true)) * 100

def smape(y_true, y_pred):
    return 2.0 * np.mean(np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true))) * 100

# 0

In [62]:
test_X,test_Xl,test_y= test0_X,test0_Xl,test0_y

bp

In [63]:
bp_pred = modelbp.predict(test_X)
bp_pred,y_true,inv_test=inv(bp_pred,test_X,test_y)

bp_mse = metrics.mean_squared_error(y_true, bp_pred)
bp_rmse = math.sqrt(mean_squared_error(y_true[:,0], bp_pred[:,0]))
bp_mae = metrics.mean_absolute_error(y_true, bp_pred)
bp_mape = mape(y_true, bp_pred)
bp_smape = smape(y_true, bp_pred)

In [64]:
print(bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape)

0.28026748 0.5294029439241177 0.26861504 262.2817039489746 76.14707350730896


dnn

In [65]:
dnn_pred = modeldnn.predict(test_X)
# dnn_pred,y_true,inv_test=inv(dnn_pred,test_year,test_X,test_y)
dnn_pred,y_true,inv_test=inv(dnn_pred,test_X,test_y)

dnn_mse = metrics.mean_squared_error(y_true, dnn_pred)
dnn_rmse = math.sqrt(mean_squared_error(y_true[:,0], dnn_pred[:,0]))
dnn_mae = metrics.mean_absolute_error(y_true, dnn_pred)
dnn_mape = mape(y_true, dnn_pred)
dnn_smape = smape(y_true, dnn_pred)

In [66]:
print(dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape)

0.26733676 0.517046183615141 0.2525518 123.32502603530884 72.38969206809998


LSTM


In [67]:
# make a prediction
lstm_pred = modellstm.predict(test_Xl)
# test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
lstm_pred,y_true,inv_test=inv(lstm_pred,test_X,test_y)
lstm_mse = metrics.mean_squared_error(y_true, lstm_pred)
lstm_rmse = math.sqrt(mean_squared_error(y_true[:,0], lstm_pred[:,0]))
lstm_mae = metrics.mean_absolute_error(y_true, lstm_pred)
lstm_mape = mape(y_true, lstm_pred)
lstm_smape = smape(y_true, lstm_pred)


In [68]:
print(lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape)

0.36926642 0.6076729552998047 0.33167157 159.7559094429016 52.04998850822449


In [69]:
bp0_mse,bp0_rmse,bp0_mae,bp0_mape,bp0_smape = bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape

In [70]:
dnn0_mse,dnn0_rmse,dnn0_mae,dnn0_mape,dnn0_smape = dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape

In [71]:
lstm0_mse,lstm0_rmse,lstm0_mae,lstm0_mape,lstm0_smape = lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape

# 1

In [72]:
test_X,test_Xl,test_y= test1_X,test1_Xl,test1_y

bp

In [73]:
bp_pred = modelbp.predict(test_X)
bp_pred,y_true,inv_test=inv(bp_pred,test_X,test_y)

bp_mse = metrics.mean_squared_error(y_true, bp_pred)
bp_rmse = math.sqrt(mean_squared_error(y_true[:,0], bp_pred[:,0]))
bp_mae = metrics.mean_absolute_error(y_true, bp_pred)
bp_mape = mape(y_true, bp_pred)
bp_smape = smape(y_true, bp_pred)

In [74]:
print(bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape)

0.2082277 0.45631972786038566 0.23045273 124.7888445854187 65.56723713874817


dnn

In [75]:
dnn_pred = modeldnn.predict(test_X)
# dnn_pred,y_true,inv_test=inv(dnn_pred,test_year,test_X,test_y)
dnn_pred,y_true,inv_test=inv(dnn_pred,test_X,test_y)

dnn_mse = metrics.mean_squared_error(y_true, dnn_pred)
dnn_rmse = math.sqrt(mean_squared_error(y_true[:,0], dnn_pred[:,0]))
dnn_mae = metrics.mean_absolute_error(y_true, dnn_pred)
dnn_mape = mape(y_true, dnn_pred)
dnn_smape = smape(y_true, dnn_pred)

In [76]:
print(dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape)

0.21928506 0.46827882291962286 0.24552512 83.66116881370544 88.1293535232544


LSTM


In [77]:
# make a prediction
lstm_pred = modellstm.predict(test_Xl)
# test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
lstm_pred,y_true,inv_test=inv(lstm_pred,test_X,test_y)
lstm_mse = metrics.mean_squared_error(y_true, lstm_pred)
lstm_rmse = math.sqrt(mean_squared_error(y_true[:,0], lstm_pred[:,0]))
lstm_mae = metrics.mean_absolute_error(y_true, lstm_pred)
lstm_mape = mape(y_true, lstm_pred)
lstm_smape = smape(y_true, lstm_pred)


In [78]:
print(lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape)

0.27395213 0.5234043623270547 0.27678186 110.98040342330933 70.23405432701111


In [79]:
bp1_mse,bp1_rmse,bp1_mae,bp1_mape,bp1_smape = bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape

In [80]:
dnn1_mse,dnn1_rmse,dnn1_mae,dnn1_mape,dnn1_smape = dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape

In [81]:
lstm1_mse,lstm1_rmse,lstm1_mae,lstm1_mape,lstm1_smape = lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape

# 2

In [82]:
test_X,test_Xl,test_y= test2_X,test2_Xl,test2_y

In [83]:
# svm_mse = metrics.mean_squared_error(y_true, svm_pred)
# # svm_rmse = math.sqrt(mean_squared_error(y_true[:,0], svm_pred[:,0]))
# svm_rmse = math.sqrt(mean_squared_error(y_true, svm_pred))
# svm_mae = metrics.mean_absolute_error(y_true, svm_pred)
# svm_mape = mape(y_true, svm_pred)
# svm_smape = smape(y_true, svm_pred)

In [84]:
# print(svm_mse,svm_rmse,svm_mae,svm_mape,svm_smape)

bp

In [85]:
bp_pred = modelbp.predict(test_X)
bp_pred,y_true,inv_test=inv(bp_pred,test_X,test_y)

bp_mse = metrics.mean_squared_error(y_true, bp_pred)
bp_rmse = math.sqrt(mean_squared_error(y_true[:,0], bp_pred[:,0]))
bp_mae = metrics.mean_absolute_error(y_true, bp_pred)
bp_mape = mape(y_true, bp_pred)
bp_smape = smape(y_true, bp_pred)

In [86]:
print(bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape)

0.10839661 0.3292364076279441 0.14166442 344.6985721588135 52.49660611152649


dnn

In [87]:
dnn_pred = modeldnn.predict(test_X)
# dnn_pred,y_true,inv_test=inv(dnn_pred,test_year,test_X,test_y)
dnn_pred,y_true,inv_test=inv(dnn_pred,test_X,test_y)

dnn_mse = metrics.mean_squared_error(y_true, dnn_pred)
dnn_rmse = math.sqrt(mean_squared_error(y_true[:,0], dnn_pred[:,0]))
dnn_mae = metrics.mean_absolute_error(y_true, dnn_pred)
dnn_mape = mape(y_true, dnn_pred)
dnn_smape = smape(y_true, dnn_pred)

In [88]:
print(dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape)

0.10306846 0.32104277519407665 0.14147884 154.3486475944519 100.11861324310303


LSTM


In [89]:
# make a prediction
lstm_pred = modellstm.predict(test_Xl)
# test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
lstm_pred,y_true,inv_test=inv(lstm_pred,test_X,test_y)
lstm_mse = metrics.mean_squared_error(y_true, lstm_pred)
lstm_rmse = math.sqrt(mean_squared_error(y_true[:,0], lstm_pred[:,0]))
lstm_mae = metrics.mean_absolute_error(y_true, lstm_pred)
lstm_mape = mape(y_true, lstm_pred)
lstm_smape = smape(y_true, lstm_pred)


In [90]:
print(lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape)

0.14495675 0.3807318645160795 0.20214678 950.0734329223633 91.74960851669312


In [91]:
# svm2_mse,svm2_rmse,svm2_mae,svm2_mape,svm2_smape = svm_mse,svm_rmse,svm_mae,svm_mape,svm_smape

In [92]:
bp2_mse,bp2_rmse,bp2_mae,bp2_mape,bp2_smape = bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape

In [93]:
dnn2_mse,dnn2_rmse,dnn2_mae,dnn2_mape,dnn2_smape = dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape

In [94]:
lstm2_mse,lstm2_rmse,lstm2_mae,lstm2_mape,lstm2_smape = lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape

# 3

svm

In [95]:
test_X,test_Xl,test_y= test3_X,test3_Xl,test3_y

In [96]:
# svm_pred = modelsvm.predict(test_X)
# svm_pred = svm_pred[:, np.newaxis]
# svm_pred,y_true,inv_test=inv(svm_pred,test_X,test_y)

In [97]:
# svm_mse = metrics.mean_squared_error(y_true, svm_pred)
# # svm_rmse = math.sqrt(mean_squared_error(y_true[:,0], svm_pred[:,0]))
# svm_rmse = math.sqrt(mean_squared_error(y_true, svm_pred))
# svm_mae = metrics.mean_absolute_error(y_true, svm_pred)
# svm_mape = mape(y_true, svm_pred)
# svm_smape = smape(y_true, svm_pred)

In [98]:
# print(svm_mse,svm_rmse,svm_mae,svm_mape,svm_smape)

bp

In [99]:
bp_pred = modelbp.predict(test_X)
bp_pred,y_true,inv_test=inv(bp_pred,test_X,test_y)

bp_mse = metrics.mean_squared_error(y_true, bp_pred)
bp_rmse = math.sqrt(mean_squared_error(y_true[:,0], bp_pred[:,0]))
bp_mae = metrics.mean_absolute_error(y_true, bp_pred)
bp_mape = mape(y_true, bp_pred)
bp_smape = smape(y_true, bp_pred)

In [100]:
print(bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape)

0.122066095 0.34937958651212314 0.16459905 5141.900253295898 112.46408224105835


dnn

In [101]:
dnn_pred = modeldnn.predict(test_X)
# dnn_pred,y_true,inv_test=inv(dnn_pred,test_year,test_X,test_y)
dnn_pred,y_true,inv_test=inv(dnn_pred,test_X,test_y)

dnn_mse = metrics.mean_squared_error(y_true, dnn_pred)
dnn_rmse = math.sqrt(mean_squared_error(y_true[:,0], dnn_pred[:,0]))
dnn_mae = metrics.mean_absolute_error(y_true, dnn_pred)
dnn_mape = mape(y_true, dnn_pred)
dnn_smape = smape(y_true, dnn_pred)

In [102]:
print(dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape)

0.1030286 0.32098068929161594 0.13114811 1446.4670181274414 112.82483339309692


LSTM


In [103]:
# make a prediction
lstm_pred = modellstm.predict(test_Xl)
# test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
lstm_pred,y_true,inv_test=inv(lstm_pred,test_X,test_y)
lstm_mse = metrics.mean_squared_error(y_true, lstm_pred)
lstm_rmse = math.sqrt(mean_squared_error(y_true[:,0], lstm_pred[:,0]))
lstm_mae = metrics.mean_absolute_error(y_true, lstm_pred)
lstm_mape = mape(y_true, lstm_pred)
lstm_smape = smape(y_true, lstm_pred)


In [104]:
print(lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape)

0.1307129 0.36154238560017343 0.17892906 3163.223457336426 114.82932567596436


In [105]:
# svm3_mse,svm3_rmse,svm3_mae,svm3_mape,svm3_smape = svm_mse,svm_rmse,svm_mae,svm_mape,svm_smape

In [106]:
bp3_mse,bp3_rmse,bp3_mae,bp3_mape,bp3_smape = bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape

In [107]:
dnn3_mse,dnn3_rmse,dnn3_mae,dnn3_mape,dnn3_smape = dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape

In [108]:
lstm3_mse,lstm3_rmse,lstm3_mae,lstm3_mape,lstm3_smape = lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape

# 4

svm

In [109]:
test_X,test_Xl,test_y= test4_X,test4_Xl,test4_y

In [110]:
# svm_pred = modelsvm.predict(test_X)
# svm_pred = svm_pred[:, np.newaxis]
# svm_pred,y_true,inv_test=inv(svm_pred,test_X,test_y)

In [111]:
# svm_mse = metrics.mean_squared_error(y_true, svm_pred)
# # svm_rmse = math.sqrt(mean_squared_error(y_true[:,0], svm_pred[:,0]))
# svm_rmse = math.sqrt(mean_squared_error(y_true, svm_pred))
# svm_mae = metrics.mean_absolute_error(y_true, svm_pred)
# svm_mape = mape(y_true, svm_pred)
# svm_smape = smape(y_true, svm_pred)

In [112]:
# print(svm_mse,svm_rmse,svm_mae,svm_mape,svm_smape)

bp

In [113]:
bp_pred = modelbp.predict(test_X)
bp_pred,y_true,inv_test=inv(bp_pred,test_X,test_y)

bp_mse = metrics.mean_squared_error(y_true, bp_pred)
bp_rmse = math.sqrt(mean_squared_error(y_true[:,0], bp_pred[:,0]))
bp_mae = metrics.mean_absolute_error(y_true, bp_pred)
bp_mape = mape(y_true, bp_pred)
bp_smape = smape(y_true, bp_pred)

In [114]:
print(bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape)

0.3221226 0.5675584583590148 0.25362864 2836.8175506591797 144.9887752532959


dnn

In [115]:
dnn_pred = modeldnn.predict(test_X)
# dnn_pred,y_true,inv_test=inv(dnn_pred,test_year,test_X,test_y)
dnn_pred,y_true,inv_test=inv(dnn_pred,test_X,test_y)

dnn_mse = metrics.mean_squared_error(y_true, dnn_pred)
dnn_rmse = math.sqrt(mean_squared_error(y_true[:,0], dnn_pred[:,0]))
dnn_mae = metrics.mean_absolute_error(y_true, dnn_pred)
dnn_mape = mape(y_true, dnn_pred)
dnn_smape = smape(y_true, dnn_pred)

In [116]:
print(dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape)

0.50405854 0.7099708021231586 0.31852165 2839.559745788574 149.6415376663208


LSTM


In [117]:
# make a prediction
lstm_pred = modellstm.predict(test_Xl)
# test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
lstm_pred,y_true,inv_test=inv(lstm_pred,test_X,test_y)
lstm_mse = metrics.mean_squared_error(y_true, lstm_pred)
lstm_rmse = math.sqrt(mean_squared_error(y_true[:,0], lstm_pred[:,0]))
lstm_mae = metrics.mean_absolute_error(y_true, lstm_pred)
lstm_mape = mape(y_true, lstm_pred)
lstm_smape = smape(y_true, lstm_pred)


In [118]:
print(lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape)

0.65739036 0.8107961248451073 0.37928727 3768.1182861328125 150.7387399673462


In [119]:
# svm4_mse,svm4_rmse,svm4_mae,svm4_mape,svm4_smape = svm_mse,svm_rmse,svm_mae,svm_mape,svm_smape

In [120]:
bp4_mse,bp4_rmse,bp4_mae,bp4_mape,bp4_smape = bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape

In [121]:
dnn4_mse,dnn4_rmse,dnn4_mae,dnn4_mape,dnn4_smape = dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape

In [122]:
lstm4_mse,lstm4_rmse,lstm4_mae,lstm4_mape,lstm4_smape = lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape

# 5

svm

In [123]:
test_X,test_Xl,test_y= test5_X,test5_Xl,test5_y

In [124]:
# svm_pred = modelsvm.predict(test_X)
# svm_pred = svm_pred[:, np.newaxis]
# svm_pred,y_true,inv_test=inv(svm_pred,test_X,test_y)

In [125]:
# svm_mse = metrics.mean_squared_error(y_true, svm_pred)
# # svm_rmse = math.sqrt(mean_squared_error(y_true[:,0], svm_pred[:,0]))
# svm_rmse = math.sqrt(mean_squared_error(y_true, svm_pred))
# svm_mae = metrics.mean_absolute_error(y_true, svm_pred)
# svm_mape = mape(y_true, svm_pred)
# svm_smape = smape(y_true, svm_pred)

In [126]:
# print(svm_mse,svm_rmse,svm_mae,svm_mape,svm_smape)

bp

In [127]:
bp_pred = modelbp.predict(test_X)
bp_pred,y_true,inv_test=inv(bp_pred,test_X,test_y)

bp_mse = metrics.mean_squared_error(y_true, bp_pred)
bp_rmse = math.sqrt(mean_squared_error(y_true[:,0], bp_pred[:,0]))
bp_mae = metrics.mean_absolute_error(y_true, bp_pred)
bp_mape = mape(y_true, bp_pred)
bp_smape = smape(y_true, bp_pred)

In [128]:
print(bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape)

0.24755064 0.49754460757797636 0.2605265 285.4926109313965 113.0400538444519


dnn

In [129]:
dnn_pred = modeldnn.predict(test_X)
# dnn_pred,y_true,inv_test=inv(dnn_pred,test_year,test_X,test_y)
dnn_pred,y_true,inv_test=inv(dnn_pred,test_X,test_y)

dnn_mse = metrics.mean_squared_error(y_true, dnn_pred)
dnn_rmse = math.sqrt(mean_squared_error(y_true[:,0], dnn_pred[:,0]))
dnn_mae = metrics.mean_absolute_error(y_true, dnn_pred)
dnn_mape = mape(y_true, dnn_pred)
dnn_smape = smape(y_true, dnn_pred)

In [130]:
print(dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape)

0.264728 0.514517258894952 0.2674108 125.8365273475647 120.99339962005615


LSTM


In [131]:
# make a prediction
lstm_pred = modellstm.predict(test_Xl)
# test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
lstm_pred,y_true,inv_test=inv(lstm_pred,test_X,test_y)
lstm_mse = metrics.mean_squared_error(y_true, lstm_pred)
lstm_rmse = math.sqrt(mean_squared_error(y_true[:,0], lstm_pred[:,0]))
lstm_mae = metrics.mean_absolute_error(y_true, lstm_pred)
lstm_mape = mape(y_true, lstm_pred)
lstm_smape = smape(y_true, lstm_pred)


In [132]:
print(lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape)

0.33600265 0.5796573538537568 0.31234175 333.16707611083984 122.9997992515564


In [133]:
# svm5_mse,svm5_rmse,svm5_mae,svm5_mape,svm5_smape = svm_mse,svm_rmse,svm_mae,svm_mape,svm_smape

In [134]:
bp5_mse,bp5_rmse,bp5_mae,bp5_mape,bp5_smape = bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape

In [135]:
dnn5_mse,dnn5_rmse,dnn5_mae,dnn5_mape,dnn5_smape = dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape

In [136]:
lstm5_mse,lstm5_rmse,lstm5_mae,lstm5_mape,lstm5_smape = lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape

# 6

svm

In [137]:
test_X,test_Xl,test_y= test6_X,test6_Xl,test6_y

In [138]:
# svm_pred = modelsvm.predict(test_X)
# svm_pred = svm_pred[:, np.newaxis]
# svm_pred,y_true,inv_test=inv(svm_pred,test_X,test_y)

In [139]:
# svm_mse = metrics.mean_squared_error(y_true, svm_pred)
# # svm_rmse = math.sqrt(mean_squared_error(y_true[:,0], svm_pred[:,0]))
# svm_rmse = math.sqrt(mean_squared_error(y_true, svm_pred))
# svm_mae = metrics.mean_absolute_error(y_true, svm_pred)
# svm_mape = mape(y_true, svm_pred)
# svm_smape = smape(y_true, svm_pred)

In [140]:
# print(svm_mse,svm_rmse,svm_mae,svm_mape,svm_smape)

bp

In [141]:
bp_pred = modelbp.predict(test_X)
bp_pred,y_true,inv_test=inv(bp_pred,test_X,test_y)

bp_mse = metrics.mean_squared_error(y_true, bp_pred)
bp_rmse = math.sqrt(mean_squared_error(y_true[:,0], bp_pred[:,0]))
bp_mae = metrics.mean_absolute_error(y_true, bp_pred)
bp_mape = mape(y_true, bp_pred)
bp_smape = smape(y_true, bp_pred)

In [142]:
print(bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape)

0.43651485 0.6606927080202659 0.3539095 123.98743629455566 122.80865907669067


dnn

In [143]:
dnn_pred = modeldnn.predict(test_X)
# dnn_pred,y_true,inv_test=inv(dnn_pred,test_year,test_X,test_y)
dnn_pred,y_true,inv_test=inv(dnn_pred,test_X,test_y)

dnn_mse = metrics.mean_squared_error(y_true, dnn_pred)
dnn_rmse = math.sqrt(mean_squared_error(y_true[:,0], dnn_pred[:,0]))
dnn_mae = metrics.mean_absolute_error(y_true, dnn_pred)
dnn_mape = mape(y_true, dnn_pred)
dnn_smape = smape(y_true, dnn_pred)

In [144]:
print(dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape)

0.52857834 0.7270339338760142 0.38934836 92.5807535648346 130.78045845031738


LSTM


In [145]:
# make a prediction
lstm_pred = modellstm.predict(test_Xl)
# test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
lstm_pred,y_true,inv_test=inv(lstm_pred,test_X,test_y)
lstm_mse = metrics.mean_squared_error(y_true, lstm_pred)
lstm_rmse = math.sqrt(mean_squared_error(y_true[:,0], lstm_pred[:,0]))
lstm_mae = metrics.mean_absolute_error(y_true, lstm_pred)
lstm_mape = mape(y_true, lstm_pred)
lstm_smape = smape(y_true, lstm_pred)


In [146]:
print(lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape)

0.7079358 0.8413892143883895 0.46398202 111.6788387298584 131.63968324661255


In [147]:
# svm6_mse,svm6_rmse,svm6_mae,svm6_mape,svm6_smape = svm_mse,svm_rmse,svm_mae,svm_mape,svm_smape

In [148]:
bp6_mse,bp6_rmse,bp6_mae,bp6_mape,bp6_smape = bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape

In [149]:
dnn6_mse,dnn6_rmse,dnn6_mae,dnn6_mape,dnn6_smape = dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape

In [150]:
lstm6_mse,lstm6_rmse,lstm6_mae,lstm6_mape,lstm6_smape = lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape

# 7

svm

In [151]:
test_X,test_Xl,test_y= test7_X,test7_Xl,test7_y

In [152]:
# svm_pred = modelsvm.predict(test_X)
# svm_pred = svm_pred[:, np.newaxis]
# svm_pred,y_true,inv_test=inv(svm_pred,test_X,test_y)

In [153]:
# svm_mse = metrics.mean_squared_error(y_true, svm_pred)
# # svm_rmse = math.sqrt(mean_squared_error(y_true[:,0], svm_pred[:,0]))
# svm_rmse = math.sqrt(mean_squared_error(y_true, svm_pred))
# svm_mae = metrics.mean_absolute_error(y_true, svm_pred)
# svm_mape = mape(y_true, svm_pred)
# svm_smape = smape(y_true, svm_pred)

In [154]:
# print(svm_mse,svm_rmse,svm_mae,svm_mape,svm_smape)

bp

In [155]:
bp_pred = modelbp.predict(test_X)
bp_pred,y_true,inv_test=inv(bp_pred,test_X,test_y)

bp_mse = metrics.mean_squared_error(y_true, bp_pred)
bp_rmse = math.sqrt(mean_squared_error(y_true[:,0], bp_pred[:,0]))
bp_mae = metrics.mean_absolute_error(y_true, bp_pred)
bp_mape = mape(y_true, bp_pred)
bp_smape = smape(y_true, bp_pred)

In [156]:
print(bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape)

0.14794813 0.3846402617579611 0.19179317 397.15802669525146 72.35853672027588


dnn

In [157]:
dnn_pred = modeldnn.predict(test_X)
# dnn_pred,y_true,inv_test=inv(dnn_pred,test_year,test_X,test_y)
dnn_pred,y_true,inv_test=inv(dnn_pred,test_X,test_y)

dnn_mse = metrics.mean_squared_error(y_true, dnn_pred)
dnn_rmse = math.sqrt(mean_squared_error(y_true[:,0], dnn_pred[:,0]))
dnn_mae = metrics.mean_absolute_error(y_true, dnn_pred)
dnn_mape = mape(y_true, dnn_pred)
dnn_smape = smape(y_true, dnn_pred)

In [158]:
print(dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape)

0.12828445 0.3581681928168708 0.16104926 243.90499591827393 80.90788722038269


LSTM


In [159]:
# make a prediction
lstm_pred = modellstm.predict(test_Xl)
# test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
lstm_pred,y_true,inv_test=inv(lstm_pred,test_X,test_y)
lstm_mse = metrics.mean_squared_error(y_true, lstm_pred)
lstm_rmse = math.sqrt(mean_squared_error(y_true[:,0], lstm_pred[:,0]))
lstm_mae = metrics.mean_absolute_error(y_true, lstm_pred)
lstm_mape = mape(y_true, lstm_pred)
lstm_smape = smape(y_true, lstm_pred)


In [160]:
print(lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape)

0.17846724 0.4224538364245682 0.2190246 398.79307746887207 61.2781822681427


In [161]:
# svm7_mse,svm7_rmse,svm7_mae,svm7_mape,svm7_smape = svm_mse,svm_rmse,svm_mae,svm_mape,svm_smape

In [162]:
bp7_mse,bp7_rmse,bp7_mae,bp7_mape,bp7_smape = bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape

In [163]:
dnn7_mse,dnn7_rmse,dnn7_mae,dnn7_mape,dnn7_smape = dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape

In [164]:
lstm7_mse,lstm7_rmse,lstm7_mae,lstm7_mape,lstm7_smape = lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape

# 8

svm

In [165]:
test_X,test_Xl,test_y= test8_X,test8_Xl,test8_y

In [166]:
# svm_pred = modelsvm.predict(test_X)
# svm_pred = svm_pred[:, np.newaxis]
# svm_pred,y_true,inv_test=inv(svm_pred,test_X,test_y)

In [167]:
# svm_mse = metrics.mean_squared_error(y_true, svm_pred)
# # svm_rmse = math.sqrt(mean_squared_error(y_true[:,0], svm_pred[:,0]))
# svm_rmse = math.sqrt(mean_squared_error(y_true, svm_pred))
# svm_mae = metrics.mean_absolute_error(y_true, svm_pred)
# svm_mape = mape(y_true, svm_pred)
# svm_smape = smape(y_true, svm_pred)

In [168]:
# print(svm_mse,svm_rmse,svm_mae,svm_mape,svm_smape)

bp

In [169]:
bp_pred = modelbp.predict(test_X)
bp_pred,y_true,inv_test=inv(bp_pred,test_X,test_y)

bp_mse = metrics.mean_squared_error(y_true, bp_pred)
bp_rmse = math.sqrt(mean_squared_error(y_true[:,0], bp_pred[:,0]))
bp_mae = metrics.mean_absolute_error(y_true, bp_pred)
bp_mape = mape(y_true, bp_pred)
bp_smape = smape(y_true, bp_pred)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  


In [170]:
print(bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape)

0.10582709 0.325310763923344 0.13873765 inf 113.35697174072266


dnn

In [171]:
dnn_pred = modeldnn.predict(test_X)
# dnn_pred,y_true,inv_test=inv(dnn_pred,test_year,test_X,test_y)
dnn_pred,y_true,inv_test=inv(dnn_pred,test_X,test_y)

dnn_mse = metrics.mean_squared_error(y_true, dnn_pred)
dnn_rmse = math.sqrt(mean_squared_error(y_true[:,0], dnn_pred[:,0]))
dnn_mae = metrics.mean_absolute_error(y_true, dnn_pred)
dnn_mape = mape(y_true, dnn_pred)
dnn_smape = smape(y_true, dnn_pred)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  


In [172]:
print(dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape)

0.099823415 0.31594843769100744 0.12749 inf 114.80640172958374


LSTM


In [173]:
# make a prediction
lstm_pred = modellstm.predict(test_Xl)
# test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
lstm_pred,y_true,inv_test=inv(lstm_pred,test_X,test_y)
lstm_mse = metrics.mean_squared_error(y_true, lstm_pred)
lstm_rmse = math.sqrt(mean_squared_error(y_true[:,0], lstm_pred[:,0]))
lstm_mae = metrics.mean_absolute_error(y_true, lstm_pred)
lstm_mape = mape(y_true, lstm_pred)
lstm_smape = smape(y_true, lstm_pred)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  


In [174]:
print(lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape)

0.13493681 0.36733746002796247 0.18365026 inf 117.17221736907959


In [175]:
# svm8_mse,svm8_rmse,svm8_mae,svm8_mape,svm8_smape = svm_mse,svm_rmse,svm_mae,svm_mape,svm_smape

In [176]:
bp8_mse,bp8_rmse,bp8_mae,bp8_mape,bp8_smape = bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape

In [177]:
dnn8_mse,dnn8_rmse,dnn8_mae,dnn8_mape,dnn8_smape = dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape

In [178]:
lstm8_mse,lstm8_rmse,lstm8_mae,lstm8_mape,lstm8_smape = lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape

# 评价指标

In [179]:
#评价指标
evaluate_rmse = pd.DataFrame(columns=['评价指标','WPD-BP','WPD-DNN','WPD-LSTM'])

In [180]:
evaluate_rmse['Test']=['站点一','站点一','站点二','站点二','站点三','站点三','站点四','站点四','站点五','站点五',
                       '站点六','站点六','站点七','站点七','站点八','站点八','站点九','站点九']
evaluate_rmse=evaluate_rmse.set_index('Test',drop=True)
evaluate_rmse['评价指标']=['RMSE','MAE','RMSE','MAE','RMSE','MAE','RMSE','MAE','RMSE','MAE','RMSE','MAE','RMSE','MAE','RMSE','MAE','RMSE','MAE',]

In [181]:
evaluate_rmse

评价指标 WPD-BP WPD-DNN WPD-LSTM
Test                              
站点一   RMSE    NaN     NaN      NaN
站点一    MAE    NaN     NaN      NaN
站点二   RMSE    NaN     NaN      NaN
站点二    MAE    NaN     NaN      NaN
站点三   RMSE    NaN     NaN      NaN
站点三    MAE    NaN     NaN      NaN
站点四   RMSE    NaN     NaN      NaN
站点四    MAE    NaN     NaN      NaN
站点五   RMSE    NaN     NaN      NaN
站点五    MAE    NaN     NaN      NaN
站点六   RMSE    NaN     NaN      NaN
站点六    MAE    NaN     NaN      NaN
站点七   RMSE    NaN     NaN      NaN
站点七    MAE    NaN     NaN      NaN
站点八   RMSE    NaN     NaN      NaN
站点八    MAE    NaN     NaN      NaN
站点九   RMSE    NaN     NaN      NaN
站点九    MAE    NaN     NaN      NaN

In [ ]:
# evaluate_rmse['SVM']=[svm0_rmse,svm1_rmse,svm2_rmse,svm3_rmse,svm4_rmse,svm5_rmse,svm6_rmse,svm7_rmse,svm8_rmse]

In [182]:
evaluate_rmse['WPD-BP']=[bp0_rmse,bp0_mae,bp1_rmse,bp1_mae,bp2_rmse,bp2_mae,bp3_rmse,bp3_mae,bp4_rmse,bp4_mae,bp5_rmse,bp5_mae,bp6_rmse,bp6_mae,bp7_rmse,bp7_mae,bp8_rmse,bp8_mae]

In [183]:
evaluate_rmse['WPD-DNN']=[dnn0_rmse,dnn0_mae,dnn1_rmse,dnn1_mae,dnn2_rmse,dnn2_mae,dnn3_rmse,dnn3_mae,dnn4_rmse,dnn4_mae,dnn5_rmse,dnn5_mae,
                      dnn6_rmse,dnn6_mae,dnn7_rmse,dnn7_mae,dnn8_rmse,dnn8_mae]

In [184]:
evaluate_rmse['WPD-LSTM']=[lstm0_rmse,lstm0_mae,lstm1_rmse,lstm1_mae,lstm2_rmse,lstm2_mae,lstm3_rmse,lstm3_mae,lstm4_rmse,lstm4_mae,lstm5_rmse,lstm5_mae,
                       lstm6_rmse,lstm6_mae,lstm7_rmse,lstm7_mae,lstm8_rmse,lstm8_mae,]

In [ ]:
# s = pd.Series([evaluate_rmse['SVM'].mean(),evaluate_rmse['BP'].mean(), evaluate_rmse['DNN'].mean(), evaluate_rmse['LSTM'].mean()], index=evaluate_rmse.columns, name='mean')
# evaluate_rmse = evaluate_rmse.append(s)

In [186]:
evaluate_rmse.to_csv('../predict/九个站点评价指标WPD.csv',index=True)
evaluate_rmse

评价指标    WPD-BP   WPD-DNN  WPD-LSTM
Test                                    
站点一   RMSE  0.529403  0.517046  0.607673
站点一    MAE  0.268615  0.252552  0.331672
站点二   RMSE  0.456320  0.468279  0.523404
站点二    MAE  0.230453  0.245525  0.276782
站点三   RMSE  0.329236  0.321043  0.380732
站点三    MAE  0.141664  0.141479  0.202147
站点四   RMSE  0.349380  0.320981  0.361542
站点四    MAE  0.164599  0.131148  0.178929
站点五   RMSE  0.567558  0.709971  0.810796
站点五    MAE  0.253629  0.318522  0.379287
站点六   RMSE  0.497545  0.514517  0.579657
站点六    MAE  0.260527  0.267411  0.312342
站点七   RMSE  0.660693  0.727034  0.841389
站点七    MAE  0.353909  0.389348  0.463982
站点八   RMSE  0.384640  0.358168  0.422454
站点八    MAE  0.191793  0.161049  0.219025
站点九   RMSE  0.325311  0.315948  0.367337
站点九    MAE  0.138738  0.127490  0.183650

# MAE

In [ ]:
#评价指标
evaluate_mae = pd.DataFrame(columns=['Test','SVM','BP','DNN','LSTM'])

In [ ]:
evaluate_mae['Test']=['0','1','2','3','4','5','6','7','8']
evaluate_mae=evaluate_mae.set_index('Test',drop=True)

In [ ]:
evaluate_mae

In [ ]:
# evaluate_mae['SVM']=[svm0_mae,svm1_mae,svm2_mae,svm3_mae,svm4_mae,svm5_mae,svm6_mae,svm7_mae,svm8_mae]

In [ ]:
evaluate_mae['BP']=[bp0_mae,bp1_mae,bp2_mae,bp3_mae,bp4_mae,bp5_mae,bp6_mae,bp7_mae,bp8_mae]

In [ ]:
evaluate_mae['DNN']=[dnn0_mae,dnn1_mae,dnn2_mae,dnn3_mae,dnn4_mae,dnn5_mae,dnn6_mae,dnn7_mae,dnn8_mae]

In [ ]:
evaluate_mae['LSTM']=[lstm0_mae,lstm1_mae,lstm2_mae,lstm3_mae,lstm4_mae,lstm5_mae,lstm6_mae,lstm7_mae,lstm8_mae]

In [ ]:
s = pd.Series([evaluate_mae['SVM'].mean(),evaluate_mae['BP'].mean(), evaluate_mae['DNN'].mean(), evaluate_mae['LSTM'].mean()], index=evaluate_mae.columns, name='mean')
evaluate_mae = evaluate_mae.append(s)

In [ ]:
# evaluate_mae.to_csv('../predict/maetest0-8.csv',index=True)
evaluate_mae

# 保存预测结果并绘图

In [ ]:
#真实值和预测值对比图
plt.plot(y_true,'royalblue',label='Real')
plt.plot(svm_pred,'orange',label='SVM')
# plt.plot(bp_pred,label='BP')
# plt.plot(dnn_pred,label='DNN')
# plt.plot(lstm_pred,label='LSTM')
plt.title('1A Trina Prediction')
plt.ylabel('Power')
plt.xlabel('Time')
plt.legend()
plt.savefig('../picture/SVM.jpg')

In [ ]:
#真实值和预测值对比图
plt.plot(y_true,'royalblue',label='Real')
# plt.plot(svm_pred,label='SVM')
plt.plot(bp_pred,'limegreen',label='BP')
# plt.plot(dnn_pred,label='DNN')
# plt.plot(lstm_pred,label='LSTM')

plt.title('1A Trina Prediction')
plt.ylabel('Power')
plt.xlabel('Time')
plt.legend()
plt.savefig('../picture/BP.jpg')

In [ ]:
#真实值和预测值对比图
plt.plot(y_true,'royalblue',label='Real')
# plt.plot(svm_pred,label='SVM')
# plt.plot(bp_pred,label='BP')
plt.plot(dnn_pred,'tomato',label='DNN')
# plt.plot(lstm_pred,label='LSTM')

plt.title('1A Trina Prediction')
plt.ylabel('Power')
plt.xlabel('Time')
plt.legend()
plt.savefig('../picture/DNN.jpg')

In [ ]:
#真实值和预测值对比图
plt.plot(y_true,'royalblue',label='Real')
# plt.plot(svm_pred,label='SVM')
# plt.plot(bp_pred,label='BP')
# plt.plot(dnn_pred,label='DNN')
plt.plot(lstm_pred,'orchid',label='LSTM')

plt.title('1A Trina Prediction')
plt.ylabel('Power')
plt.xlabel('Time')
plt.legend()
plt.savefig('../picture/LSTM.jpg')

# 保存

In [ ]:
#真实值和预测值对比图
plt.plot(y_true,label='Real')
plt.plot(svm_pred,label='SVM')
plt.plot(bp_pred,label='BP')
plt.plot(dnn_pred,label='DNN')
plt.plot(lstm_pred,label='LSTM')

plt.title('1A Trina Prediction')
plt.ylabel('Power')
plt.xlabel('Time')
plt.legend()
plt.savefig('../picture/SVMLSTM.jpg')

#训练集和测试集 预测结果保存
predict_ = pd.DataFrame(inv_test)
# predict_.columns = ['Year', 'Month', 'Day', 'Hour', 'Minutes', 'WTC', 'WRH', 'GHR', 'DHR',
#        'RGT', 'RDT', '1A Trina_power']
predict_.columns = ['Month(t-1)', 'Day(t-1)', 'Hour(t-1)', 'Minutes(t-1)', 'WTC(t-1)',
       'WRH(t-1)', 'GHR(t-1)', 'DHR(t-1)', 'WD(t-1)', 'WDR(t-1)', 'RGT(t-1)',
       'RDT(t-1)', '1A Trina_power(t-1)', '1A Trina_power(t)',
       '1A Trina_power(t+1)']
predict_['SVM'] = svm_pred
predict_['BP'] = bp_pred
predict_['DNN'] = dnn_pred
predict_['LSTM'] = lstm_pred
predict_.to_csv('../predict/1Atest_pred_-WDWDR_svm&bp&dnn.csv',index=False)


画出例如2020年1月20号的预测图

In [ ]:
predict_.columns

In [ ]:
# from scipy.interpolate import spline
predict_[['Month(t-1)', 'Day(t-1)', 'Hour(t-1)', 'Minutes(t-1)']] = predict_[['Month(t-1)', 'Day(t-1)', 'Hour(t-1)', 'Minutes(t-1)']].astype("int")

# Day0120 = predict_.loc[(predict_['Year'] == 2020) & (predict_['Month'] == ) & (predict_['Day'] == 20),:]
Day0120 = predict_.loc[(predict_['Month(t-1)'] == 2 ) & (predict_['Day(t-1)'] == 20),:]
#real_power_day.set_index('time', inplace=False)
Real=Day0120.set_index('Hour(t-1)', inplace=False)['1A Trina_power(t+1)']
svm_pr=Day0120.set_index('Hour(t-1)', inplace=False)['SVM']
bp_pr=Day0120.set_index('Hour(t-1)', inplace=False)['BP']
dnn_pr=Day0120.set_index('Hour(t-1)', inplace=False)['DNN']
lstm_pr=Day0120.set_index('Hour(t-1)', inplace=False)['LSTM']

fig = plt.figure()
plt.plot(Real,label='Real')
plt.plot(svm_pr,label='SVM')
plt.plot(bp_pr,label='BP')
plt.plot(dnn_pr,label='DNN')
plt.plot(lstm_pr,label='LSTM')

plt.title('Prediction 2020-2-20')
plt.ylabel('Power')
plt.xlabel('Time')
# plt.legend(['Train', 'Test'], loc='upper left')
plt.legend()
plt.savefig('../picture/2-20pred.jpg')
#ax = fig.add_subplot(1,1,1)
#ax.plot(real_power_day_)
# real_power_day_.plot()
#real_power_day_.set_axis(['00:15:00','06:15:00','12:15:00','18:15:00'])
#real_power_day_

# 2020 

In [ ]:
# from scipy.interpolate import spline
predict_[['Month(t-1)', 'Day(t-1)', 'Hour(t-1)', 'Minutes(t-1)']] = predict_[['Month(t-1)', 'Day(t-1)', 'Hour(t-1)', 'Minutes(t-1)']].astype("int")

# Day0120 = predict_.loc[(predict_['Year'] == 2020) & (predict_['Month'] == ) & (predict_['Day'] == 20),:]
Day0120 = predict_.loc[(predict_['Month(t-1)'] == 3 ) & (predict_['Day(t-1)'] == 25),:]
#real_power_day.set_index('time', inplace=False)
Real=Day0120.set_index('Hour(t-1)', inplace=False)['1A Trina_power(t+1)']
svm_pr=Day0120.set_index('Hour(t-1)', inplace=False)['SVM']
bp_pr=Day0120.set_index('Hour(t-1)', inplace=False)['BP']
dnn_pr=Day0120.set_index('Hour(t-1)', inplace=False)['DNN']
lstm_pr=Day0120.set_index('Hour(t-1)', inplace=False)['LSTM']

fig = plt.figure()
plt.plot(Real,label='Real')
plt.plot(svm_pr,label='SVM')
plt.plot(bp_pr,label='BP')
plt.plot(dnn_pr,label='DNN')
plt.plot(lstm_pr,label='LSTM')

plt.title('Prediction 2020-3-25')
plt.ylabel('Power')
plt.xlabel('Time')
# plt.legend(['Train', 'Test'], loc='upper left')
plt.legend()
plt.savefig('../picture/3-25pred.jpg')
#ax = fig.add_subplot(1,1,1)
#ax.plot(real_power_day_)
# real_power_day_.plot()
#real_power_day_.set_axis(['00:15:00','06:15:00','12:15:00','18:15:00'])
#real_power_day_